In [28]:
import math
import random

In [29]:
class Neuron():
    def __init__(self,position_in_layer,is_output_neuron=False):
        self.weights=[]
        self.inputs=[]
        self.output=None
        self.updated_weights=[]   #helpful for backpropogation update
        self.is_output_neuron=is_output_neuron
        self.delta=None           #used in backpropogation for update
        self.position_in_layer=position_in_layer
        
    def attach_to_output(self,neurons):
        self.output_neurons=neurons
        
    
    def sigmoid(self, x):
        return (1 / (1 + math.exp(-x)))
    
    
    def init_weights(self, num_input):    #helps in initializing weights[currently done randomly using the random function]
        for i in range(num_input+1):
            self.weights.append(random.uniform(0,1))   
            
            
    def predict(self, row):
        self.inputs = []
        activation = 0
        for weight, feature in zip(self.weights, row):
            self.inputs.append(feature)
            activation = activation + (weight*feature)
        self.output = self.sigmoid(activation)
        return self.output    
            
    def update_neuron(self):
        self.weights = []
        for new_weight in self.updated_weights:
            self.weights.append(new_weight)
            
    def calculate_update(self, learning_rate, target):
        if self.is_output_neuron:
            self.delta = (self.output - target)*self.output*(1-self.output)
        else:
            delta_sum = 0
            cur_weight_index = self.position_in_layer             # this is to know which weights this neuron is contributing in the output layer
            for output_neuron in self.output_neurons:
                delta_sum = delta_sum + (output_neuron.delta * output_neuron.weights[cur_weight_index])

            self.delta = delta_sum*self.output*(1-self.output)              # Update this neuron delta
            
        self.updated_weights = []   #dont have to overwrite the weights as earlier ones has to be used for other updates in backpropagation
        
        # Iterate over each weight and update them
        for cur_weight, cur_input in zip(self.weights, self.inputs):
            gradient = self.delta*cur_input
            new_weight = cur_weight - learning_rate*gradient
            self.updated_weights.append(new_weight)        
           

In [30]:
class Layer():
    def __init__(self, num_neuron, is_output_layer = False):
        self.is_output_layer = is_output_layer
        self.neurons = []
        for i in range(num_neuron):                         # Will create that much neurons in the layer
            neuron = Neuron(i,  is_output_neuron=is_output_layer)
            self.neurons.append(neuron)
    
    def attach(self, layer):
        for in_neuron in self.neurons:
            in_neuron.attach_to_output(layer.neurons)
            
    def init_layer(self, num_input):
        for neuron in self.neurons:
            neuron.init_weights(num_input)
    
    def predict(self, row):
        row.append(1) 
        activations = [neuron.predict(row) for neuron in self.neurons]
        return activations

In [31]:
class MultiLayerPerceptron():          #will be creating the multi-layer perceptron with only two layer only

    def __init__(self, learning_rate = 0.01, num_iteration = 100):
        self.layers = []
        self.learning_rate = learning_rate
        self.num_iteration = num_iteration
        
        
    def add_output_layer(self, num_neuron):
        self.layers.insert(0, Layer(num_neuron, is_output_layer = True))
    
    def add_hidden_layer(self, num_neuron):
        hidden_layer = Layer(num_neuron) # Created an hidden layer
        hidden_layer.attach(self.layers[0])  # Attached the last added layer to this new layer
        self.layers.insert(0, hidden_layer)  # Added this layer to the whole network
        
    def update_layers(self, target):
        for layer in reversed(self.layers):
            for neuron in layer.neurons:
                neuron.calculate_update(self.learning_rate, target)  
        for layer in self.layers:
            for neuron in layer.neurons:
                neuron.update_neuron()
    
    def fit(self, X, y):
        num_row = len(X)
        num_feature = len(X[0]) #We assume that we have a rectangular matrix
        
        # Initialized the weights throughout each of the layer
        self.layers[0].init_layer(num_feature)
        
        for i in range(1, len(self.layers)):
            num_input = len(self.layers[i-1].neurons)
            self.layers[i].init_layer(num_input)

        #Running the algorithm
        for i in range(self.num_iteration):
            # Stochastic Gradient Descent
            r_i = random.randint(0,num_row-1)
            row = X[r_i] # taking the random sample from the dataset
            yhat = self.predict(row)
            target = y[r_i]
            
            # Updated the layers using backpropagation   
            self.update_layers(target)
            
            # on every 100 iteration we calculate the error on the whole training set
            if i % 1000 == 0:
                total_error = 0
                for r_i in range(num_row):
                    row = X[r_i]
                    yhat = self.predict(row)
                    error = (y[r_i] - yhat)
                    total_error = total_error + error**2
                mean_error = total_error/num_row
                print(f"Itr {i} with error = {mean_error}")
        
    
    def predict(self, row):        
        activations = self.layers[0].predict(row)
        for i in range(1, len(self.layers)):
            activations = self.layers[i].predict(activations)

        outputs = []
        for activation in activations:                        # For having the output either 1 or 0
            if activation >= 0.5:
                outputs.append(1.0)
            else:
                outputs.append(0.0)
                           
        return outputs[0]

In [32]:
X = [[0,0], [0,1], [1,0], [1,1]]                   #Random boolean function
y = [0, 0, 1, 0]

a= MultiLayerPerceptron(learning_rate = 0.1, num_iteration = 20000) #parameters for the network

a.add_output_layer(num_neuron = 1)
a.add_hidden_layer(num_neuron = 3)
a.add_hidden_layer(num_neuron = 2)

a.fit(X,y)

Itr 0 with error = 0.75
Itr 1000 with error = 0.25
Itr 2000 with error = 0.25
Itr 3000 with error = 0.25
Itr 4000 with error = 0.25
Itr 5000 with error = 0.25
Itr 6000 with error = 0.25
Itr 7000 with error = 0.25
Itr 8000 with error = 0.25
Itr 9000 with error = 0.25
Itr 10000 with error = 0.25
Itr 11000 with error = 0.25
Itr 12000 with error = 0.25
Itr 13000 with error = 0.0
Itr 14000 with error = 0.0
Itr 15000 with error = 0.0
Itr 16000 with error = 0.0
Itr 17000 with error = 0.0
Itr 18000 with error = 0.0
Itr 19000 with error = 0.0
